---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [13]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    #loop line by line through the college towns file
    university_towns = []
    state = ''
    line = ''
    
    #towns_file = pd.read_table("university_towns.txt")
    with open('university_towns.txt') as file:
        for line in file:
            #print(line)
            if '[edit]' in line:
                # Remember this `state` until the next is found
                state = line[:line.find('[')]
                #print(state)
            else:
                # Otherwise, we have a city; keep `state` as last-seen
                if ' (' in line:
                    line = line[:line.find(' (')]
                    #print(line)
                else: 
                    if '\n' in line:
                        line = line[:line.find('\n')]
                university_towns.append([state, line])

    towns_df = pd.DataFrame(university_towns, columns=["State", "RegionName"])
    
    return towns_df


In [17]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''

    gdp_quarters = pd.read_excel('gdplev.xls', skiprows=219, names=['Quarter', 'GDP Billions'], usecols=[4,5])
    gdp_quarters.set_index('Quarter')
    recession_start = ''
    for i in (gdp_quarters.index-2):

        #print(i, gdp_quarters.iloc[i]['Quarter'], gdp_quarters.iloc[i][ 'GDP Billions'], gdp_quarters.iloc[i+1][ 'GDP Billions'], gdp_quarters.iloc[i+2][ 'GDP Billions'] )

        q = gdp_quarters.iloc[i]['Quarter']
        gi = gdp_quarters.iloc[i][ 'GDP Billions']
        gi1 = gdp_quarters.iloc[i+1][ 'GDP Billions']
        gi2 = gdp_quarters.iloc[i+2][ 'GDP Billions']
        if ((gi > gi1) and (gi1 > gi2) and (recession_start == '')):
            recession_start = q

    return recession_start

In [18]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    gdp_quarters = pd.read_excel('gdplev.xls', skiprows=219, names=['Quarter', 'GDP Billions'], usecols=[4,5])
    #gdp_quarters.set_index('Quarter')

    recession_start = get_recession_start()
    recession_started = False
    recession_ended = False
    recession_end = ''

    for i in gdp_quarters.index:
        if (recession_start == gdp_quarters.iloc[i-2]['Quarter']):
            recession_started = True
        if ((recession_started == True) and (recession_ended == False)):
            gi = gdp_quarters.iloc[i][ 'GDP Billions']
            gi1 = gdp_quarters.iloc[i-1][ 'GDP Billions']
            gi2 = gdp_quarters.iloc[i-2][ 'GDP Billions']
            if ((gi>gi1) and (gi1>gi2)):
                recession_ended = True
                recession_end = gdp_quarters.iloc[i]['Quarter']
                #print(recession_start, recession_end)
            
    return recession_end

In [19]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    gdp_quarters = pd.read_excel('gdplev.xls', skiprows=219, names=['Quarter', 'GDP Billions'], usecols=[4,5])

    recession_start = get_recession_start()
    recession_bottomed = False
    recession_started = False

    recession_bottom = ''
    for i in gdp_quarters.index:
        if ((recession_started == True) and (recession_bottomed == False)):
            if (gdp_quarters.iloc[i+1]['GDP Billions'] > gdp_quarters.iloc[i]['GDP Billions']):
                recession_bottom = gdp_quarters.iloc[i]['Quarter']
                recession_bottomed = True
        if (gdp_quarters.iloc[i]['Quarter'] == recession_start):
            recession_started = True

    #print(recession_bottom)
    
    return recession_bottom

In [25]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    housingdata = pd.read_csv('City_Zhvi_AllHomes.csv')

    #convert two-letter-state to full name of state
    housingdata['State'] = housingdata['State'].map(states)
    
    #set index to state, regionname (multiindex)
    housingdata.set_index(["State","RegionName"], inplace=True)

    #filter columns by year, only want 2000 to 2016
    housingdata = housingdata.filter(regex='^20', axis=1)
    
    #group select columns by quarter, calculates average per quarter
    housingdata = housingdata.groupby(pd.PeriodIndex(housingdata.columns, freq='Q'), axis=1).mean()


    return housingdata

In [26]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()
    university_towns = get_list_of_university_towns()
    housingdata = convert_housing_data_to_quarters().dropna()

    #get ratio (column) of housing prices between recession start and bottom into multiindex dataframe
    hdf = housingdata.copy()
    ratio = pd.DataFrame({'ratio': hdf[recession_start].div(hdf[recession_bottom])})

    #now need to compare university towns against others, so split
    hdf.columns = hdf.columns.to_series().astype(str)
    hdf = pd.concat([hdf, ratio], axis=1)
    hdf = pd.DataFrame(hdf)
    hdf.reset_index(['State','RegionName'], inplace = True)

    utown_ratio = hdf.loc[list(university_towns.index)]['ratio'].dropna()

    # minus/exclude university towns from total dataframe to get non-university towns
    nonutown_ratio_index = set(hdf.index) - set(utown_ratio)

    #and then calculate the ratio
    nonutown_ratio = hdf.loc[list(nonutown_ratio_index),:]['ratio'].dropna()

    # now run ttest and get trhe requested tuple
    tstat, p = tuple(ttest_ind(utown_ratio, nonutown_ratio))
    #print(tstat, p)

    #return tuple where different = True or False, return p-value, and whether university-town is better or not
    different = p < 0.05
    result = tstat < 0
    better = ["university town", "non-university town"]
    #print(different)
    #print(result)
    #print(better[result])

    return (different, p, better[result])